# 분류
- 일봉 기준, 비트코인 업/다운 분류.
- 3개의 부스팅 분류모델활용 : LGBM, xgboost,catboost
- 주가의 상승/하락을 예측하는 분류
- voting ensemble하여 의견종합

# 플로우

1. 부스팅모델은 2차원 인풋데이터활용 - 데이터 전처리
2. 모델학습
3. 피쳐엔지니어링 - 성능개선
4. 하이퍼파라미터튜닝 - 옵튜나, kfold 
5. 모델 재학습 , 콜백 저장 / 로드
6. 검증
7. 결과 및 개선방향

# 데이터 수집 및 전처리

- 비트코인 데이터받기
- 더 많은 지표(피쳐)넣기
- 종가 제거 -> 0 , 1로 변환 -> 마지막 행은 제거(다음날종가를 모르니까)

- 더 많은 피처를 넣어 학습데이터생성, 종가를 제거하고 0,1의 타겟밸류를 추가

In [49]:
# 데이터 받는 법 - ccxt라이브러리
# 다양하지 않으므로 PASS

# symbol = "BTC/USDT"
# ohlcv = exchange.fetch_ohlcv(symbol=symbol, timeframe='1d', limit=1500, params={})
# # 데이터 프레임 생성
# df_c = pd.DataFrame(ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
# datetime_proccessing(df_c)

AuthenticationError: binance {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action."}

In [50]:
# 데이터로드
import ccxt 
import pprint
import csv
import time
import pandas as pd

# API key load
def read_API_URL_CSV():
    file_path = '/Users/hwangseokjun/Desktop/All/2023/자동트레이딩/게만아(클래스101/최종결과물/APIkey.csv'
    lines_to_read = [2, 4, 6]
    data = {}
    with open(file_path, newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        for i, row in enumerate(csv_reader, start=1):
            if i in lines_to_read: # data에 인덱스 i를 키로 하고, 현재 행의 첫 번째 열(row[0])을 값으로 저장
                data[i] = row[0]
    return data

data = read_API_URL_CSV() # dict : [1] : access, [2] : secret
api_key = data[2]
api_secret = data[4]

exchange = ccxt.binance(config={
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True
    }
)

## 데이터 정보
- Interval : 일봉이므로 일정한 값으로 증가(24시간)
- openTime : 최초 거래시작 시간
- closeTime : 마지막 거래 시간
- takerVolume, tradeCount : 즉시 긁는 시장가매매볼륨,시장가매매횟수  
: 코인 시장의 활발함, 유동성, 그리고 투자자들의 관심도를 평가하는 데 유용한 정보를 제공
- limit = 1000이라 3회에 걸쳐 로드 후, merge


In [102]:
# 데이터 로드  -  API klines(binance API docs)

import requests
import pandas as pd

# 실수. 소수점 두째자리까지 표시
pd.options.display.float_format = '{:.2f}'.format

def get_data(symbol, interval, start_time=None, end_time=None, limit=None):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time,
        "limit": limit
    }
    res = requests.get(url, params=params)
    data = res.json()
    df = pd.DataFrame(data, 
                      columns=[
                          'datetime', 
                          'open', 'high', 'low', 'close', 'volume',
                          'interval','tradeCount','takerVolume','amount',
                          'openTime','closeTime'
                      ])
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)
    df = df.round(2)
    return df

# 날짜 전처리 (unix -> datetime)
# Unix : 1970년 1월 1일 00:00:00 UTC(협정 세계시)부터 경과한 밀리초의 총 수
def datetime_proccessing(df):
    pd_ts = pd.to_datetime(df['datetime'], utc=True, unit='ms')     # unix timestamp to pandas Timeestamp
    pd_ts = pd_ts.dt.tz_convert("Asia/Seoul")                       # convert timezone
    pd_ts = pd_ts.dt.tz_localize(None)
    df['datetime'] = pd_ts
    # df.set_index(pd_ts, inplace=True)
    # df = df.drop(['datetime'], axis=1)
    return df

# 서울시간 -> Unix밀리초로 변환
def convert_to_utc_unix(dt_str):
    # 한국 시간대로 Timestamp 객체 생성
    kst_time = pd.Timestamp(dt_str, tz='Asia/Seoul')
    utc_time = kst_time.tz_convert('UTC')
    # 밀리초 단위 Unix 시간스탬프로 변환
    unix_timestamp = int(utc_time.timestamp()) * 1000
    return unix_timestamp


end_time = "2024-01-27 09:00:00"
timestamp = convert_to_utc_unix(end_time)
df = get_data("BTCUSDT", "1h", end_time=timestamp, limit=1000)

print(df.info())
df



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   datetime     1000 non-null   int64  
 1   open         1000 non-null   float64
 2   high         1000 non-null   float64
 3   low          1000 non-null   float64
 4   close        1000 non-null   float64
 5   volume       1000 non-null   float64
 6   interval     1000 non-null   float64
 7   tradeCount   1000 non-null   float64
 8   takerVolume  1000 non-null   float64
 9   amount       1000 non-null   float64
 10  openTime     1000 non-null   float64
 11  closeTime    1000 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 93.9 KB
None


,datetime,open,high,low,close,volume,interval,tradeCount,takerVolume,amount,openTime,closeTime
0,1702717200000,42179.13,42284.00,42179.12,42209.49,1031.12,1702720799999.00,43551905.02,45067.00,610.79,25796420.75,0.00
1,1702720800000,42209.48,42261.10,42191.10,42261.09,481.41,1702724399999.00,20327813.44,46475.00,255.99,10809381.43,0.00
2,1702724400000,42261.08,42339.55,42221.10,42331.99,608.07,1702727999999.00,25712132.92,41919.00,347.56,14697099.39,0.00
3,1702728000000,42332.00,42490.00,42266.86,42418.96,1141.95,1702731599999.00,48402434.10,49005.00,559.09,23698067.84,0.00
4,1702731600000,42418.96,42548.00,42243.00,42396.10,1660.67,1702735199999.00,70425679.58,76786.00,755.82,32047119.86,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1706299200000,42064.48,42246.82,41838.83,41977.18,2276.20,1706302799999.00,95635287.33,69073.00,1150.32,48334108.82,0.00
996,1706302800000,41977.18,42048.00,41878.93,41998.78,1148.22,1706306399999.00,48185348.52,37238.00,636.86,26722173.18,0.00
997,1706306400000,41998.77,42011.75,41820.64,41906.28,1674.96,1706309999999.00,70176183.23,40348.00,634.84,26601315.23,0.00
998,1706310000000,41906.28,41972.00,41800.02,41823.51,674.43,1706313599999.00,28247748.55,30967.00,301.98,12648596.21,0.00


In [103]:
# Add more data
# 코드 실행시, 위에 셀을 실행하여 df를 다시만들고 실행하기 !

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

def load_data(first_df):
    df_list = [first_df]
    max_iterations = 100  # 최대 반복 횟수

    for _ in range(max_iterations):
        last_df = df_list[-1]  # 마지막으로 추가된 df 참조
        end_time = last_df.iloc[0, 0]
        df = get_data("BTCUSDT", "1h", end_time=end_time, limit=1000)
        
        if df.iloc[0, 0] == last_df.iloc[0, 0]:  # 새로운df와 직전df의 첫째날이 같다면 break
            break
        
        df_list.append(df)
    
    # 리스트 각 요소에 할당하려면 enumerate필요
    for idx, df in enumerate(df_list):
        df_list[idx] = datetime_proccessing(df)

    return df_list

df_list = load_data(df)
print(len(df_list))
print(df_list[:1])
print(df_list[-2:])

# 2017-08-17 ~ 2024-01-27

57
[               datetime     open     high      low    close  volume  \
0   2023-12-16 18:00:00 42179.13 42284.00 42179.12 42209.49 1031.12   
1   2023-12-16 19:00:00 42209.48 42261.10 42191.10 42261.09  481.41   
2   2023-12-16 20:00:00 42261.08 42339.55 42221.10 42331.99  608.07   
3   2023-12-16 21:00:00 42332.00 42490.00 42266.86 42418.96 1141.95   
4   2023-12-16 22:00:00 42418.96 42548.00 42243.00 42396.10 1660.67   
..                  ...      ...      ...      ...      ...     ...   
995 2024-01-27 05:00:00 42064.48 42246.82 41838.83 41977.18 2276.20   
996 2024-01-27 06:00:00 41977.18 42048.00 41878.93 41998.78 1148.22   
997 2024-01-27 07:00:00 41998.77 42011.75 41820.64 41906.28 1674.96   
998 2024-01-27 08:00:00 41906.28 41972.00 41800.02 41823.51  674.43   
999 2024-01-27 09:00:00 41823.51 41866.67 41708.00 41785.94  930.04   

            interval  tradeCount  takerVolume  amount    openTime  closeTime  
0   1702720799999.00 43551905.02     45067.00  610.79 25796420.7

# Merge & 피쳐엔지니어링

In [107]:
# Merge
df_merged = pd.concat(df_list).drop_duplicates(subset='datetime').sort_values(by='datetime').reset_index(drop=True)
df_merged

,datetime,open,high,low,close,volume,interval,tradeCount,takerVolume,amount,openTime,closeTime
0,2017-08-17 13:00:00,4261.48,4313.62,4261.32,4308.83,47.18,1502945999999.00,202366.14,171.00,35.16,150952.48,0.00
1,2017-08-17 14:00:00,4308.83,4328.69,4291.37,4315.32,23.23,1502949599999.00,100304.82,102.00,21.45,92608.28,0.00
2,2017-08-17 15:00:00,4330.29,4345.45,4309.37,4324.35,7.23,1502953199999.00,31282.31,36.00,4.80,20795.32,0.00
3,2017-08-17 16:00:00,4316.62,4349.99,4287.41,4349.99,4.44,1502956799999.00,19241.06,25.00,2.60,11291.35,0.00
4,2017-08-17 17:00:00,4333.32,4377.85,4333.32,4360.69,0.97,1502960399999.00,4239.50,28.00,0.81,3552.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
56360,2024-01-27 05:00:00,42064.48,42246.82,41838.83,41977.18,2276.20,1706302799999.00,95635287.33,69073.00,1150.32,1970-01-01 22:25:34.108820,0.00
56361,2024-01-27 06:00:00,41977.18,42048.00,41878.93,41998.78,1148.22,1706306399999.00,48185348.52,37238.00,636.86,1970-01-01 16:25:22.173180,0.00
56362,2024-01-27 07:00:00,41998.77,42011.75,41820.64,41906.28,1674.96,1706309999999.00,70176183.23,40348.00,634.84,1970-01-01 16:23:21.315230,0.00
56363,2024-01-27 08:00:00,41906.28,41972.00,41800.02,41823.51,674.43,1706313599999.00,28247748.55,30967.00,301.98,1970-01-01 12:30:48.596210,0.00


In [105]:
# 컬럼 파악하기
# openTime, closeTime

def unix_proccessing(df, column):
    pd_ts = pd.to_datetime(df[column], utc=True, unit='ms')    
    pd_ts = pd_ts.dt.tz_convert("Asia/Seoul")                       # convert timezone
    pd_ts = pd_ts.dt.tz_localize(None)
    df[column] = pd_ts
    return df

column = 'openTime'
df_test = unix_proccessing(df, column)
display(df_test)


,datetime,open,high,low,close,volume,interval,tradeCount,takerVolume,amount,openTime,closeTime
0,2023-12-16 18:00:00,42179.13,42284.00,42179.12,42209.49,1031.12,1702720799999.00,43551905.02,45067.00,610.79,1970-01-01 16:09:56.420750,0.00
1,2023-12-16 19:00:00,42209.48,42261.10,42191.10,42261.09,481.41,1702724399999.00,20327813.44,46475.00,255.99,1970-01-01 12:00:09.381430,0.00
2,2023-12-16 20:00:00,42261.08,42339.55,42221.10,42331.99,608.07,1702727999999.00,25712132.92,41919.00,347.56,1970-01-01 13:04:57.099390,0.00
3,2023-12-16 21:00:00,42332.00,42490.00,42266.86,42418.96,1141.95,1702731599999.00,48402434.10,49005.00,559.09,1970-01-01 15:34:58.067840,0.00
4,2023-12-16 22:00:00,42418.96,42548.00,42243.00,42396.10,1660.67,1702735199999.00,70425679.58,76786.00,755.82,1970-01-01 17:54:07.119860,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-01-27 05:00:00,42064.48,42246.82,41838.83,41977.18,2276.20,1706302799999.00,95635287.33,69073.00,1150.32,1970-01-01 22:25:34.108820,0.00
996,2024-01-27 06:00:00,41977.18,42048.00,41878.93,41998.78,1148.22,1706306399999.00,48185348.52,37238.00,636.86,1970-01-01 16:25:22.173180,0.00
997,2024-01-27 07:00:00,41998.77,42011.75,41820.64,41906.28,1674.96,1706309999999.00,70176183.23,40348.00,634.84,1970-01-01 16:23:21.315230,0.00
998,2024-01-27 08:00:00,41906.28,41972.00,41800.02,41823.51,674.43,1706313599999.00,28247748.55,30967.00,301.98,1970-01-01 12:30:48.596210,0.00


In [108]:
# 1차 : 노이즈 제거 - 주가와 관련없는 시간데이터
drop_col = ['interval',
           'openTime',
           'closeTime']
org_df_merged = df_merged.copy()
df_merged = df_merged.drop(columns = drop_col)
df_merged

,datetime,open,high,low,close,volume,tradeCount,takerVolume,amount
0,2017-08-17 13:00:00,4261.48,4313.62,4261.32,4308.83,47.18,202366.14,171.00,35.16
1,2017-08-17 14:00:00,4308.83,4328.69,4291.37,4315.32,23.23,100304.82,102.00,21.45
2,2017-08-17 15:00:00,4330.29,4345.45,4309.37,4324.35,7.23,31282.31,36.00,4.80
3,2017-08-17 16:00:00,4316.62,4349.99,4287.41,4349.99,4.44,19241.06,25.00,2.60
4,2017-08-17 17:00:00,4333.32,4377.85,4333.32,4360.69,0.97,4239.50,28.00,0.81
...,...,...,...,...,...,...,...,...,...
56360,2024-01-27 05:00:00,42064.48,42246.82,41838.83,41977.18,2276.20,95635287.33,69073.00,1150.32
56361,2024-01-27 06:00:00,41977.18,42048.00,41878.93,41998.78,1148.22,48185348.52,37238.00,636.86
56362,2024-01-27 07:00:00,41998.77,42011.75,41820.64,41906.28,1674.96,70176183.23,40348.00,634.84
56363,2024-01-27 08:00:00,41906.28,41972.00,41800.02,41823.51,674.43,28247748.55,30967.00,301.98


# 타겟밸류 생성 & Split
- 총 2가지 실험
 1. 분류모델이니 기본적인 업/다운 (이진분류)
 2. 낮은상승률 1, 높은상승률 10, 낮은하락률 2, 높은하락률 20 으로 분류 (4가지 분류)

In [109]:
from sklearn.model_selection import train_test_split

In [110]:
# target 생성 : 이진분류 업(1)/다운(0)
def make_target(df):
    targets = []
    for i in range(len(df)-1):
    
        if df['close'].iloc[i+1] > df['close'].iloc[i]:
            targets.append(1)
        else:
            targets.append(0)
    return targets

target_col = make_target(df_merged)
target_col.append(1) # 마지막행의 다음은 상승했음 24.01.27 10:00:00
print(len(target_col))

df_merged['up_down'] = target_col
display(df_merged)

# Data Split 1 ( Train : Test  =  8 : 2 )
x = df_merged.iloc[:,:-1]
y = df_merged.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

# Data Split 2 ( Train : Varidation = 8 : 2 )
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size = 0.2, random_state = 42)
print(len(x_train),len(x_val), len(x_test))


56365


,datetime,open,high,low,close,volume,tradeCount,takerVolume,amount,up_down
0,2017-08-17 13:00:00,4261.48,4313.62,4261.32,4308.83,47.18,202366.14,171.00,35.16,1
1,2017-08-17 14:00:00,4308.83,4328.69,4291.37,4315.32,23.23,100304.82,102.00,21.45,1
2,2017-08-17 15:00:00,4330.29,4345.45,4309.37,4324.35,7.23,31282.31,36.00,4.80,1
3,2017-08-17 16:00:00,4316.62,4349.99,4287.41,4349.99,4.44,19241.06,25.00,2.60,1
4,2017-08-17 17:00:00,4333.32,4377.85,4333.32,4360.69,0.97,4239.50,28.00,0.81,1
...,...,...,...,...,...,...,...,...,...,...
56360,2024-01-27 05:00:00,42064.48,42246.82,41838.83,41977.18,2276.20,95635287.33,69073.00,1150.32,1
56361,2024-01-27 06:00:00,41977.18,42048.00,41878.93,41998.78,1148.22,48185348.52,37238.00,636.86,0
56362,2024-01-27 07:00:00,41998.77,42011.75,41820.64,41906.28,1674.96,70176183.23,40348.00,634.84,0
56363,2024-01-27 08:00:00,41906.28,41972.00,41800.02,41823.51,674.43,28247748.55,30967.00,301.98,0


36073 9019 11273


# 모델학습 및 검증
- lgbm,xgboost,catboost
- 하이퍼파라미터 튜닝없이 일단 학습 & 검증 후,
- 피쳐엔지니어링을 통해 최상의 피쳐조합 생성


In [ ]:
from xgboost.sklearn import XGBClassifier, XGBRegressor
import xgboost as xgb
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor
from lightgbm import early_stopping, log_evaluation
from catboost import CatBoostClassifier, CatBoostRegressor
from catboost import Pool

In [ ]:
# lgbm_params = {
#     ...
#     'device': 'gpu',
#     'gpu_device_id': 0,  # 사용할 GPU ID를 선택합니다.
#     ...
# }

LGBM = LGBMClassifier(device='gpu'),  # LightGBM GPU 설정
XGB = XGBClassifier(tree_method='gpu_hist'),  # XGBoost GPU 설정
Cat = CatBoostClassifier(verbose=0, task_type='GPU'),  # CatBoost GPU 설정

In [ ]:
org_model_dic = {
    "LGBM": LGBM,
    "XGB" : XGB,
    "Cat": Cat
}

class Models_learning_evaluation:
    def __init__(self, model_dict):
        self.model_dict = model_dict
        self.trained_models = {}
        
    # Learning each Models : train , val models
    def model_learning(self):
        total_len = len(self.model_dict)
        for idx, (model_name, model) in enumerate(self.model_dict.items()):
            print(f"[{idx + 1}/{total_len}] model_{model_name} Learning ...")
            self.trained_models[f"model_{model_name}"] = model.fit(x_train, y_train)
            print(f"model_{model_name} Completed !!!")
            
    # Evaluation All
    def evaluation(self):
        total_len = len(self.trained_models)
        for idx, (model_name, model) in enumerate(self.trained_models.items(), start = 1):
            print(f"\n\n[{idx}/{total_len}] {model_name} Prediction !")
            train_pred = model.predict(x_train)
            val_pred = model.predict(x_val)
            train_score = evaluation_metric(y_train, train_pred, squared = False)
            val_score = evaluation_metric(y_val, val_pred, squared = False)
            print("Train score %.4f" % train_score)
            print("Validation score %.4f" % val_score)

# evaluation_metric = mean_squared_log_error
# evaluator = Models_learning_evaluation(org_model_dict)
# evaluator.model_learning()
# evaluator.evaluation()

# # Globalizing : Models dic
# trained_models = evaluator.trained_models
# print(f"\n{trained_models}")